# TITLE IMAGE GOES HERE

---

# Project Summary


## Project Description


## Project Goals


## Key Findings




---

# Imports and Set-Up

In [1]:
#Imports

#Common Python Libraries
import pandas as pd
import numpy as np

---

# Data Acquisition

> **Approach:**
>
> The data...

**Takeaways:**

---

# Data Preparation

> **Approach:**
>
> ...

**Takeaways:**

---

# Exploration

> **Approach:**
>
> ...

## Univariate Exploration

**Univariate Takeaways & Questions:**

---

## Split the Data into Train, Validate, and Test

---

## Bivariate Exploration

**Bivariate Takeaways & Questions:**

---

## Multivariate Exploration

**Multivariate Takeaways:**

---

## Statistical Testing

**Takeaways:**

---

# Modeling

> **Approach:**
>
> ...

### Scaling

---

### Establish a Baseline Model

**Takeaway:**

---

### Model 1

### Model 2

### Model 3

**Takeaways:**

---

# Additional Analysis

---

# Conclusion and Next Steps